In [2]:
import pandas as pd
import glob
import os

# Dataframe zur Analyse
os.chdir = '/bundestag_klimaschutz/'
all_files = glob.glob("*.csv")

file_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    file_list.append(df)

df = pd.concat(file_list, axis=0, ignore_index=True)

In [3]:
df.drop('Jens Ackermann;"FDP";1975;"Sachsen-Anhalt";;"2. Juli 1975";"http://www.jens-ackermann.info";"jens.ackermann@bundestag.de"', axis=1, inplace=True)

In [4]:
df.drop(df.index[:1], inplace=True)